![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/test-specific-notebooks/Fairness_Demo.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest on John Snow Labs

In [ ]:
!pip install "langtest[johnsnowlabs,transformers]"

# Harness and its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [1]:
#Import Harness from the LangTest library
from langtest import Harness


It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>



| Parameter     | Description |
| - | - |
| **task**      | Task for which the model is to be evaluated (text-classification or ner) |
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |


<br/>
<br/>

# Fairness Testing

Fairness testing is a critical aspect of evaluating the performance of a machine learning model, especially when the model has potential implications for specific groups of people. Fairness testing aims to ensure that the model is not biased towards or against any particular group and that it produces unbiased results for all groups.
To support fairness testing, several fairness tests are available, which evaluate the model's performance on various attributes such as gender.

**`Supported Fairness tests :`**<br>

- **`min_gender_f1_score`**: Determine if any gender(male, female or unknown) has less than the desired f1 score.

- **`max_gender_f1_score`**:  Determine if any gender(male, female or unknown) has more than the desired f1 score.


<br/>
<br/>




## Testing fairness of a pretrained NER model/pipeline

Testing a model's fairness gives us an idea on how our model performs on different types of input text.

We can directly pass a pretrained model/pipeline from hub as the model parameter in harness and run the tests.

### Test Configuration

Test configuration can be passed in the form of a YAML file as shown below or using .configure() method


**Config YAML format** :
```
tests:     
  defaults:
    min_pass_rate: 0.65
  fairness:
    min_gender_f1_score:
      min_score: 0.66  
    max_gender_f1_score:
      max_score:
        male: 0.99
        female: 0.95
```

If config file is not present, we can also use the **.configure()** method to manually configure the harness to perform the needed tests.


In [2]:
harness = Harness(task='ner', model= {"model": "ner.dl", "hub": "johnsnowlabs"})

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]
Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  },
  "fairness": {
   "min_gender_f1_score": {
    "min_score": 0.6
   }
  },
  "representation": {
   "min_label_representation_count": {
    "min_count": 50
   }
  }
 }
}


We can use the .configure() method to manually configure the tests we want to perform.

In [3]:
harness.configure({
  'tests': {
    'defaults': {'min_pass_rate':0.5},
    'fairness': {
      'min_gender_f1_score': {'min_score': 0.75},
    }
  }
})

{'tests': {'defaults': {'min_pass_rate': 0.5},
  'fairness': {'min_gender_f1_score': {'min_score': 0.75}}}}

Here we have configured the harness to perform two bias tests (replace_to_female_pronouns and replace_to_hindu_names) and defined the minimum pass rate for each test.


### Generating the test cases.




In [4]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


harness.generate() method automatically generates the test cases (based on the provided configuration)

In [5]:
harness.testcases()

,category,test_type,test_case
0,fairness,min_gender_f1_score,male
1,fairness,min_gender_f1_score,female
2,fairness,min_gender_f1_score,unknown


harness.testcases() method gives the produced test cases in form of a pandas data frame.

### Running the tests

In [6]:
harness.run()

Running testcases... :   0%|          | 0/3 [00:00<?, ?it/s]

Running testcases... : 100%|██████████| 3/3 [00:09<00:00,  3.29s/it]


Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

In [7]:
harness.generated_results()

,category,test_type,test_case,expected_result,actual_result,pass
0,fairness,min_gender_f1_score,male,0.75,0.917577,True
1,fairness,min_gender_f1_score,female,0.75,0.871345,True
2,fairness,min_gender_f1_score,unknown,0.75,0.963128,True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

### Report of the tests

In [8]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,fairness,min_gender_f1_score,0,3,100%,50%,True


Called after harness.run() and it summarizes the results giving information about pass and fail counts and overall test pass/fail flag.